In [16]:
from scipy.special import expit #to calculate logit 
import scipy.io as si
import numpy as np

#.mat file has two matrix X (5000*400) and y(5000*1)
File = si.loadmat('ex4data1.mat')

#sigmoid function implementation
def sigmoid_function(z):
          return expit(z)

#sigmoid gradient implementation
def sigmoid_gradient(Z):
    temp = sigmoid_function(Z)
    return (temp*(1 - temp))

#implement Forward Propagation
def compute_input_layer(theta1,input_layer):
    return (sigmoid_function(np.dot(input_layer,theta1.T)))

def compute_hidden_layer(theta2,A):
    return (sigmoid_function(np.dot(A,theta2.T)))

#return 5000*10 matrix
def compute_hypothesis(Theta1,Theta2,input_layer):
    
    #get 5000*25 matrix
    A1 = compute_input_layer(Theta1,input_layer)
    
    #change A dim to 5000*26
    A1 = np.insert(A1,0,1,axis = 1)

    #get 5000*10 matrix
    A2 = compute_hidden_layer(Theta2,A1)
    return A2 , A1

#implement Cost function for Neural Nertwork with no Regularization

#return the actual result of Kth classifier for a given data in training set
def get_result(actual_result,K,data):
    if actual_result[data] == K:
        return 1
    else:
        return 0

#return the value for Kth classifier for a given data in training set
def get_hypo_value_K(hypothesis,data,K):
        return (hypothesis[data].item(K))

#implement Cost function for Neural Nertwork with  Regularization
def Cost_function_reg(t_theta1,t_theta2,input_layer,actual_result,K,Lambda):
    
    #create theta1 and theta2
    theta1 = t_theta1
    theta1 = np.delete(theta1,0,1)
    theta2 = t_theta2
    theta2 = np.delete(theta2,0,1)
    
    #calculate hypothesis
    hypothesis1 , hypothesis2 = compute_hypothesis(t_theta1,t_theta2,input_layer)
    
    cost = 0
    #iterate it for each training data
    for i in range(len(hypothesis1)):
        #for each class
        for j in range(K):
            term1 =  - get_result(actual_result,j,i)*np.log(get_hypo_value_K(hypothesis1,i,j)) 
            term2 =  - (1 - get_result(actual_result,j,i))*np.log(1 - get_hypo_value_K(hypothesis1,i,j)) 
            cost = cost + term1 + term2
            
    cost = (cost/(len(hypothesis1)))
    
    #calculate Regularizarion Term
    
    #minimize each theta presents in all_theta dictionary

    #regularize theta1
    t1 = np.sum(theta1*theta1) 
    
    #regularize theta2
    t2 = np.sum(theta2*theta2)
    
    Reg_term = (Lambda/(2*(len(hypothesis1)))) * (t1 + t2)
   
    #update cost
    cost = cost + Reg_term
    return cost


In [24]:
#load input_layer
input_layer = np.array(File['X'])
input_layer = np.insert(input_layer,0,1,axis = 1)

#create actual result of training set in form of {0,1}
def result_binary(actual_result,num_labels):
    y = np.zeros((len(actual_result),num_labels))
    for i in range(len(actual_result)):
        temp = actual_result[i]
        y[i][temp - 1] = 1
    return y

output_layer = result_binary(File['y'],10)

#randomly initializes theata1(25,401) and theta2(10,26) for a given epsilon


epsilon = 0.12
init_theta1 = np.random.uniform(-epsilon,epsilon,(25,401))
init_theta2 = np.random.uniform(-epsilon,epsilon,(10,26))

In [9]:
#implement BackPropagation
def BackPropagation_Reg(init_theta1,init_theta2,input_layer,output_layer,Lambda):
    
    t1 = init_theta1
    t2 = init_theta2
    
    #calculate forward propagation
    hypothesis1 , hypothesis2 = compute_hypothesis(init_theta1,init_theta2,input_layer)
    
    #computes small delta
    
    #compute delta_3
    delta_3 = hypothesis1 - output_layer
    
    #compute delta_2
    term1 = np.dot(delta_3,init_theta2).T
    term1 = np.delete(term1,0,0)
    delta_2 = np.multiply(term1.T,sigmoid_gradient(np.dot(input_layer,init_theta1.T)))
    
    #compute gradient for layer 1 with regularization
    grad_1 = np.dot(input_layer.T,delta_2).T
    grad_1 = grad_1/len(hypothesis1)
    
    t1 = np.delete(t1,0,1)
    t1 = np.insert(t1,0,0,1)
    
    grad_1 = grad_1 + (Lambda/len(hypothesis1))*t1
    
    #compute gradient for layer 2
    grad_2 = np.dot(hypothesis2.T,delta_3)
    grad_2 = np.delete(grad_2,0,0).T
    grad_2 = grad_2/len(hypothesis2)
    
    t2 = np.delete(t2,0,1)
    grad_2 = grad_2 + (Lambda/len(hypothesis1))*t2
    
    
    return (grad_1 , grad_2)


In [3]:
#perform  Gradient Checking on a small neural network

#make a small neural network

#define sizes
input_layer_size = 3 #including bias unit
hidden_layer_size = 5 #including bias unit
num_labels = 3
m = 5

#put some random values
#randomly  initializes test_theta's with a range of (-.12,.12)
epsilon = 0.12
test_theta1 = np.random.uniform(-epsilon,epsilon,(hidden_layer_size -1,input_layer_size ))
test_theta2 = np.random.uniform(-epsilon,epsilon,(num_labels,hidden_layer_size ))

#randomly generate input layer (5*2)
test_input_layer = np.random.rand(m,input_layer_size - 1)
test_input_layer = np.insert(test_input_layer,0,1,axis = 1)
print(test_input_layer)

#randomly generate  actual result 
test_result = np.random.randint(0,3,(m,1))
print(test_result)

#create test actualresult in binary form
test_actual_result = np.zeros((len(test_result),num_labels))
for i in range(len(test_result)):
         test_actual_result[i][test_result[i]] = 1

print(test_actual_result)

#make a long test_THETA VECTOR 
test_theta_L = np.concatenate((test_theta1.flatten(),test_theta2.flatten()))
print(test_theta1.shape)
print(test_theta2.shape)
print(test_theta_L)

[[ 1.          0.2016731   0.64745088]
 [ 1.          0.23133295  0.08664872]
 [ 1.          0.56538822  0.15139938]
 [ 1.          0.38616217  0.97634764]
 [ 1.          0.40109214  0.23699668]]
[[0]
 [0]
 [2]
 [1]
 [1]]
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]
(4, 3)
(3, 5)
[ 0.0258074   0.11338325 -0.04833079 -0.119952   -0.11695457 -0.00280994
 -0.03501438  0.03343907  0.09152103  0.09980646 -0.00840855  0.01665259
  0.05156353  0.00033068 -0.05419041  0.10510945  0.11694273  0.06280476
  0.03257324  0.07114657  0.07815911 -0.02362363  0.03707092  0.04472612
  0.02385372  0.03645262 -0.03674317]


In [18]:
def gradient_checking(test_theta_L,epsilon,num_labels,Lambda):
    GRADIENT_CHECK = np.zeros((len(test_theta_L),1))
    for i in range(len(test_theta_L)):
        inc = np.zeros((len(test_theta_L),1))
        inc[i] = inc[i] + epsilon
    
        theta_plus = test_theta_L + inc
        theta_minus = test_theta_L - inc
    
        #unrolling for theta_plus
        pass_theta1 = np.array(theta_plus[0:12]).reshape(4,3)
        pass_theta2 = np.array(theta_plus[12:]).reshape(3,5)
    
        cost_plus = Cost_function_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels,Lambda)
    
        #unrolling for theta_minus
        pass_theta1 = np.array(theta_minus[0:12]).reshape(4,3)
        pass_theta2 = np.array(theta_minus[12:]).reshape(3,5)
    
        cost_minus = Cost_function_reg(pass_theta1,pass_theta2,test_input_layer,test_result,num_labels,Lambda)
    
        GRADIENT_CHECK[i] = (cost_plus - cost_minus)/(2*epsilon)
    
    return GRADIENT_CHECK


In [22]:
Lambda = 3
#make calls to backpropagation and Cost_function for small neural network
TEST_GRADIENT_1 , TEST_GRADIENT_2 = BackPropagation_Reg(test_theta1,test_theta2,test_input_layer,test_actual_result,Lambda)

test_theta_L = test_theta_L.reshape(len(test_theta1.flatten()) + len(test_theta2.flatten()) ,1)
GRADIENT_CHECK = gradient_checking(test_theta_L,.0001,num_labels,Lambda)
#COMPARE OUTPUT OF GRADIENT_CHECKING AND BACKPROPAGATION

GRADIENT_CHECK1 = np.array(GRADIENT_CHECK[0:12]).reshape(4,3)
GRADIENT_CHECK2 = np.array(GRADIENT_CHECK[12:]).reshape(3,5)
GRADIENT_CHECK2 = np.delete(GRADIENT_CHECK2,0,1)

print("output from Backpropagtion for theta 1 \n")
print(TEST_GRADIENT_1)
print("\noutput from Gradient Checking for theta 1 \n")
print(GRADIENT_CHECK1)

print("\noutput from Backpropagtion for theta 2 \n")
print(TEST_GRADIENT_2)
print("\noutput from Gradient Checking for theta 2 \n")
print(GRADIENT_CHECK2)

output from Backpropagtion for theta 1 

[[ 0.00465677  0.06911329 -0.02704999]
 [ 0.00245254 -0.07055993 -0.00194372]
 [ 0.00907769  0.02411864  0.05830718]
 [ 0.00024239 -0.00285066  0.01065836]]

output from Gradient Checking for theta 1 

[[ 0.00465677  0.06911329 -0.02704999]
 [ 0.00245254 -0.07055993 -0.00194372]
 [ 0.00907769  0.02411864  0.05830718]
 [ 0.00024239 -0.00285066  0.01065836]]

output from Backpropagtion for theta 2 

[[ 0.07072487  0.02797388  0.13217175  0.14127553]
 [ 0.08894143  0.10505101  0.11294098  0.05639876]
 [ 0.18739889  0.16132361  0.18270869  0.14523484]]

output from Gradient Checking for theta 2 

[[ 0.07072487  0.02797388  0.13217175  0.14127553]
 [ 0.08894143  0.10505101  0.11294098  0.05639876]
 [ 0.18739889  0.16132361  0.18270869  0.14523484]]
